In [1]:
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv(r"data\export.csv")

In [9]:
df

,clean_id,raw_id,clinic_application_date,intake_date,group,signing_date,sheet,therapy_start_date,therapy_end_date,has_source_CAU,...,N,Eligibility,Randomized,Dropout,Research Dropout,Clinical Dropout,In Waiting List,Finished,Active,Not Cooperative
0,329822563,329822563,2025-03-16 00:00:00,2025-03-16 00:00:00,CAU,2025-03-16 00:00:00,נשירה מחקרית,2025-09-02 00:00:00,0,False,...,True,True,True,True,True,False,False,False,False,False
1,a0017,a0017s,2025-03-12 00:00:00,2025-04-06 00:00:00,NaN,0,פספוסי גיוסים,0,0,False,...,True,False,False,False,False,False,False,False,False,False
2,a0226,a0226s,2025-01-07 00:00:00,2025-01-14 00:00:00,NaN,0,אי התאמה למחקר,0,0,False,...,True,False,False,False,False,False,False,False,False,False
3,a0531,a0531s,2025-01-21 00:00:00,2025-02-06 00:00:00,NaN,0,אי הסכמה למחקר,0,0,False,...,True,True,False,False,False,False,False,False,False,False
4,a0556,a0556s,2025-04-20 00:00:00,2025-05-11 00:00:00,NaN,0,פספוסי גיוסים,0,0,False,...,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,y9000,y9000s,2024-09-23 00:00:00,2025-01-14 00:00:00,NaN,0,אי התאמה למחקר,2025-04-22 00:00:00,2025-05-27 00:00:00,False,...,True,False,False,False,False,False,False,False,False,False
297,y9212,y9212,2025-03-25 00:00:00,2025-03-25 00:00:00,CAU,2025-03-25 00:00:00,משתתפים פעילים,0,0,False,...,True,True,True,False,False,False,True,False,True,False
298,y9411,y9411s,2024-12-16 00:00:00,2025-03-20 00:00:00,Stepped Care,0,נשירה קלינית- לאחר ת. טיפול,0,2025-09-09 00:00:00,False,...,True,True,True,True,False,True,False,False,False,False
299,z1585,z1585,2025-02-27 00:00:00,2025-04-20 00:00:00,Stepped Care,2025-04-20 00:00:00,משתתפים פעילים,0,0,False,...,True,True,True,False,False,False,True,False,True,False


In [3]:
consort_groups = [
    
    'N',
    'Eligibility',
    'Randomized',
    'Dropout',
    'Research Dropout',
    'Clinical Dropout',
    'In Waiting List',
    'Finished',
    'Active',
    'Not Cooperative',

    ]

In [4]:
df.clean_id.nunique() == df.shape[0]

True

In [5]:
df['Eligibility']

0       True
1      False
2      False
3       True
4      False
       ...  
296    False
297     True
298     True
299     True
300    False
Name: Eligibility, Length: 301, dtype: bool

In [6]:
results = []

for i in consort_groups:
    row = {
        'consort_group': i,
        'size': df[i].sum(),
        'participants': df[df[i]].clean_id,   
    }
    results.append(row)

In [7]:
df.group.unique()

array(['CAU', nan, 'Stepped Care', 'טיפול בקבוצת הורים '], dtype=object)

In [8]:
group_results = []

for group in ['CAU', 'Stepped Care', None]:
    if group is None:
        grouped_df = df[~df.group.isin(['CAU', 'Stepped Care'])]
    else:
        grouped_df = df[df.group == group]
        
    print(group, grouped_df.shape)
    for i in consort_groups:
        row = {
            'consort_group': i,
            'size': grouped_df[i].sum(),
            'participants': grouped_df[grouped_df[i]].clean_id,  
            'experiment_group': group
        }
        group_results.append(row)

CAU (56, 32)
Stepped Care (49, 32)
None (196, 32)


In [9]:
cau = [i for i in group_results if i['experiment_group'] == 'CAU']
ipc = [i for i in group_results if i['experiment_group'] == 'Stepped Care']
non = [i for i in group_results if i['experiment_group'] is None]

In [11]:
pd.DataFrame(results).to_csv("consort_group_sizes.csv", index=0)

In [12]:
pd.DataFrame(cau).to_csv("consort_groups_sizes_CAU.csv", index=0)

In [13]:
pd.DataFrame(ipc).to_csv("consort_group_sizes_Stepped_Care.csv", index=0)

In [15]:
pd.DataFrame(non).to_csv("consort_sizes_no_group.csv", index=0)

In [16]:
start = "first_contact_date"
end = 'therapy_starting_date'

In [17]:
(df[start] == '0').value_counts()
    
    
(df[end] == '0').value_counts()

therapy_starting_date
True     262
False     39
Name: count, dtype: int64

In [20]:
ended_df = df[(df[end] != '0') & (df[start] != '0')][['clean_id', 'group', start, end]]

In [21]:
ended_df['waiting_duration'] =  pd.to_datetime(ended_df[end]) - pd.to_datetime(ended_df[start])

In [19]:
ended_df.to_csv("patients_with_started_therapy.csv")

In [22]:
ended_df.to_csv("waiting_time_duration.csv")

In [ ]:
'CAU', 'Stepped Care'

In [41]:
ended_df_cau = ended_df[ended_df.group == 'CAU']

In [42]:
ended_df_ipc = ended_df[ended_df.group == 'Stepped Care']

In [43]:
ended_df_no_group =  ended_df[~ended_df.group.isin(['CAU', 'Stepped Care'])]

In [44]:
pd.to_datetime(ended_df_cau[end]) - pd.to_datetime(ended_df_cau[start])

0      170 days
101    600 days
104    187 days
110    989 days
184   1379 days
230     35 days
241     19 days
248      0 days
253     93 days
276     54 days
290    201 days
dtype: timedelta64[ns]

In [45]:
pd.to_datetime(ended_df_ipc[end]) - pd.to_datetime(ended_df_ipc[start])

8      39 days
42     54 days
91    328 days
114    35 days
144   843 days
155   127 days
167    74 days
181    73 days
228    84 days
244    62 days
254   219 days
259    80 days
262   188 days
282    79 days
289   166 days
dtype: timedelta64[ns]

In [46]:
pd.to_datetime(ended_df_no_group[end]) - pd.to_datetime(ended_df_no_group[start])

19    271 days
26     20 days
87    101 days
93    202 days
115   468 days
134   187 days
159    51 days
162    27 days
225   204 days
296   211 days
dtype: timedelta64[ns]

In [7]:
df.columns

Index(['clean_id', 'raw_id', 'clinic_application_date', 'intake_date', 'group',
       'signing_date', 'sheet', 'therapy_start_date', 'therapy_end_date',
       'has_source_CAU', 'has_source_IPC-SSC', 'has_source_אי הסכמה למחקר',
       'has_source_אי התאמה למחקר', 'has_source_אין שת"פ טיפולי',
       'has_source_משתתפים פעילים', 'has_source_נשירה מחקרית',
       'has_source_נשירה קלינית- לאחר ת. טיפול', 'has_source_סיימו טיפול',
       'has_source_עלייה לרמה 2', 'has_source_פספוסי גיוסים',
       'first_contact_date', 'therapy_starting_date', 'N', 'Eligibility',
       'Randomized', 'Dropout', 'Research Dropout', 'Clinical Dropout',
       'In Waiting List', 'Finished', 'Active', 'Not Cooperative'],
      dtype='object')